In [1]:
import pandas as pd
import numpy as np
import os
import sys
import torch
from torch.utils.data import Dataset, DataLoader

sys.path.append('../../')
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from src.data.data_utils import *
from src.data.dataset import *

/u/ayang1/.conda/envs/CropTS/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
data_root = '/scratch/bbug/ayang1/raw_data/lucas/s1_lucas_2018'
save_path = '/scratch/bbug/ayang1/datasets/lucas_dataset_large/'

# Collect ee generated time series data
# ds = [
#     'S1_point_10days_10m_1Jan-31Dec_Ireland_ratio-db.csv',
#     'S1_point_10days_10m_1Jan-31Dec_Italy_ratio-db.csv',
#     'S1_point_10days_10m_1Jan-31Dec_Netherlands_ratio-db.csv',
#     'S1_point_10days_10m_1Jan-31Dec_Portugal_ratio-db.csv',
#     'S1_point_10days_10m_1Jan-31Dec_Belgium_ratio-db.csv',
#     'S1_point_10days_10m_1Jan-31Dec_Bulgaria_ratio-db.csv',
#     'S1_point_10days_10m_1Jan-31Dec_Hungary_ratio-db.csv',
# ]

ds = ['S1_point_10days_10m_1Jan-31Dec_Slovenia_ratio-db.csv']



In [30]:

# Lucas labels 
labels = pd.read_csv('/scratch/bbug/ayang1/raw_data/lucas/lucas_2018/copernicus_filtered/lucas_2018_filtered.csv')

dfs = []
for path in ds:
    dfs.append(pd.read_csv(os.path.join(data_root, path)))

data= pd.concat(dfs)

/tmp/ipykernel_1669866/2164346621.py:2: DtypeWarning: Columns (34,39,44) have mixed types. Specify dtype option on import or set low_memory=False.
  labels = pd.read_csv('/scratch/bbug/ayang1/raw_data/lucas/lucas_2018/copernicus_filtered/lucas_2018_filtered.csv')


In [31]:
# Add labels to signals based on point id
crop_data = add_lucas_labels(data, labels)
crop_data.drop('system:index', axis=1, inplace=True)
print(np.unique(crop_data['LABEL'], return_counts=True))
crop_data = crop_data.loc[crop_data['LABEL']!='NOT_CROP']

Loading files
Creating dataset of size 20707


100%|██████████| 20707/20707 [00:05<00:00, 3768.62it/s]


(array(['B11', 'B15', 'B16', 'B18', 'B51', 'B52', 'B55', 'NOT_CROP'],
      dtype=object), array([  125,     4,   234,    69,     8,   100,   126, 20041]))


In [24]:
# Filter out classes with less than 1000 samples
counts = np.unique(crop_data['LABEL'], return_counts=True)
to_drop = [counts[0][i] for i in range(len(counts[0])) if counts[1][i] < 1000]
# crop_data = crop_data.loc[~crop_data['LABEL'].isin(to_drop)]

print('Dropped classes:', to_drop)
print(len(crop_data))

Dropped classes: ['B11', 'B15', 'B16', 'B18', 'B51', 'B52', 'B55']
666


In [5]:
# crop_data.to_csv(os.path.join(save_path, ''))
crop_data = pd.read_csv(os.path.join(save_path, 'lucas_large.csv'))
crop_data.drop(['Unnamed: 0', 'POINT_ID'], axis=1, inplace=True)

In [ ]:
crop_data.head()

,POINT_ID,VHVV_20180101,VHVV_20180111,VHVV_20180121,VHVV_20180131,VHVV_20180210,VHVV_20180220,VHVV_20180302,VHVV_20180312,VHVV_20180322,...,VV_20181008,VV_20181018,VV_20181028,VV_20181107,VV_20181117,VV_20181127,VV_20181207,VV_20181217,.geo,LABEL
6396,46682550,0.161721,0.153333,0.138939,0.153570,0.132272,0.263998,0.220625,0.163274,0.145163,...,-10.978528,-12.056416,-10.774232,-9.553301,-9.440448,-10.040397,-11.787124,-12.719151,"{""type"":""MultiPoint"",""coordinates"":[]}",B11
6397,46682550,0.136804,0.118816,0.120690,0.139844,0.125986,0.257054,0.200515,0.120664,0.139629,...,-11.474958,-10.978122,-10.350073,-8.732989,-9.447535,-9.617530,-11.726073,-11.282050,"{""type"":""MultiPoint"",""coordinates"":[]}",B11
6398,46682550,0.200223,0.123761,0.162866,0.162130,0.185823,0.263060,0.173952,0.106127,0.089005,...,-10.308127,-9.005195,-9.401302,-8.783696,-8.736219,-9.261295,-11.213137,-10.753014,"{""type"":""MultiPoint"",""coordinates"":[]}",B11
6399,46682550,0.186205,0.130751,0.144223,0.180731,0.220439,0.300093,0.217380,0.131332,0.108336,...,-9.953787,-8.617062,-8.267812,-8.489451,-6.921855,-8.259746,-9.347631,-8.492420,"{""type"":""MultiPoint"",""coordinates"":[]}",B11
6400,46682550,0.182978,0.160903,0.219331,0.178556,0.084817,0.099239,0.217228,0.097813,0.153357,...,-11.015224,-11.433616,-9.907618,-10.787681,-9.382615,-11.181036,-11.785312,-12.185155,"{""type"":""MultiPoint"",""coordinates"":[]}",B11


In [6]:
# Save datasets as npy files
arr = crop_data.to_numpy()
# vhvv = arr[:, np.newaxis, 0:36]
vh = arr[:, np.newaxis, 36:72].astype(np.float32)
vv = arr[:, np.newaxis, 72:108].astype(np.float32)
labels = arr[:, 109].astype(str)

data = np.concatenate([vh, vv], axis=1)

In [13]:
# Generate masks
seq_len = vv.shape[2]
masks = []
for i in range(data.shape[0]):
    mask = np.ones(seq_len)
    mask[np.random.randint(seq_len):] = 0
    masks.append(mask)

[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]),
 array([1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]),
 array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [16]:
train_signals, test_signals, train_labels, test_labels = train_test_split(data, labels, test_size=0.3, random_state=42)
train_signals, val_signals, train_labels, val_labels = train_test_split(train_signals, train_labels, test_size=0.1, random_state=42)

In [17]:
train_masks = np.array(masks[:train_signals.shape[0]])
val_masks = np.array(masks[train_signals.shape[0]:train_signals.shape[0]+val_signals.shape[0]])
test_masks = np.array(masks[train_signals.shape[0]+val_signals.shape[0]:])

assert train_labels.shape[0] == train_masks.shape[0] == train_signals.shape[0]
assert val_labels.shape[0] == val_masks.shape[0] == val_signals.shape[0]
assert test_labels.shape[0] == test_masks.shape[0] == test_signals.shape[0]

assert len(np.unique(train_labels)) == len(np.unique(val_labels)) == len(np.unique(test_labels))

In [18]:
print(len(train_labels), len(val_labels), len(test_labels))

62528 6948 29776


In [20]:
# np.save(os.path.join(save_path, 'train_signals.npy'), train_signals, allow_pickle=True)
# np.save(os.path.join(save_path, 'train_labels.npy'), train_labels, allow_pickle=True)
np.save(os.path.join(save_path, 'vh_vv_full/train_masks.npy'), train_masks, allow_pickle=True)

# np.save(os.path.join(save_path, 'val_signals.npy'), val_signals, allow_pickle=True)
# np.save(os.path.join(save_path, 'val_labels.npy'), val_labels, allow_pickle=True)
np.save(os.path.join(save_path, 'vh_vv_full/val_masks.npy'), val_masks, allow_pickle=True)

# np.save(os.path.join(save_path, 'test_signals.npy'), test_signals, allow_pickle=True) 
# np.save(os.path.join(save_path, 'test_labels.npy'), test_labels, allow_pickle=True)
np.save(os.path.join(save_path, 'vh_vv_full/test_masks.npy'), test_masks, allow_pickle=True)

In [3]:
ds = CropTypeDataset(save_path+'vh_vv_full/', 'train', include_masks=True)

In [6]:
dataloader = DataLoader(ds, batch_size=7)